In [225]:
import pandas as pd
import numpy as np
from datetime import datetime

SOURCE = 'SYSTEM-01'
KEYS = ['policyID']

curr_df = pd.read_csv('curr.csv', dtype=str, sep=',')
curr_df.head()

,policyID,statecode,county,eq_site_limit,hu_site_limit,fl_site_limit,fr_site_limit,tiv_2011,tiv_2012,eq_site_deductible,hu_site_deductible,fl_site_deductible,fr_site_deductible,point_latitude,point_longitude,line,construction,point_granularity
0,119736,FL,CLAY COUNTY,498960,498960,498960,498960,498960,792148.9,0,9979.2,0,0,30.102261,-81.711777,Residential,Masonry,1
1,448094,FL,CLAY COUNTY,1322376.3,1322376.3,1322376.3,1322376.3,1322376.3,1438163.57,0,0,0,0,30.063936,-81.707664,Residential,Masonry,3
2,206893,FL,CLAY COUNTY,190724.4,190724.4,190724.4,190724.4,190724.4,192476.78,0,0,0,0,30.089579,-81.700455,Residential,Wood,1
3,333743,FL,CLAY COUNTY,0,79520.76,0,0,79520.76,86854.48,0,0,0,0,30.063236,-81.707703,Residential,Wood,3
4,172534,FL,CLAY COUNTY,0,254281.5,0,254281.5,254281.5,246144.49,0,0,0,0,30.060614,-81.702675,Residential,Wood,1


In [226]:
last_df = None
try:
    last_df = pd.read_csv('last.csv', dtype=str, sep=',')
except:
    last_df = pd.DataFrame(columns=curr_df.columns.tolist(), dtype=str,)
    
last_df.head()

,policyID,statecode,county,eq_site_limit,hu_site_limit,fl_site_limit,fr_site_limit,tiv_2011,tiv_2012,eq_site_deductible,hu_site_deductible,fl_site_deductible,fr_site_deductible,point_latitude,point_longitude,line,construction,point_granularity
0,119736,FL,CLAY COUNTY,498960,498960,498960,498960,498960,792148.9,0,9979.2,0,0,30.102261,-81.711777,Residential,Masonry,1
1,448094,FL,CLAY COUNTY,1322376.3,1322376.3,1322376.3,1322376.3,1322376.3,1438163.57,0,0,0,0,30.063936,-81.707664,Residential,Masonry,3
2,206893,FL,CLAY COUNTY,190724.4,190724.4,190724.4,190724.4,190724.4,192476.78,0,0,0,0,30.089579,-81.700455,Residential,Wood,1
3,333743,FL,CLAY COUNTY,0,79520.76,0,0,79520.76,86854.48,0,0,0,0,30.063236,-81.707703,Residential,Wood,3
4,172534,FL,CLAY COUNTY,0,254281.5,0,254281.5,254281.5,246144.49,0,0,0,0,30.060614,-81.702675,Residential,Wood,1


In [227]:
def _get_stamped_capture(registers_df, capture_type, capture_source):
    
    registers_df['capture_type'] = capture_type
    registers_df['capture_source'] = capture_source    
    registers_df['capture_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    return registers_df

In [228]:
def get_wide_inserts_df(curr_df, last_df, keys, source):
    
    inserts_df = pd.merge(curr_df, last_df, on=keys, how='outer', indicator=True)
    inserts_df = inserts_df[inserts_df['_merge'] == 'left_only'].reset_index(drop=True)
    inserts_df = inserts_df[keys]

    inserts_df = pd.merge(inserts_df, curr_df, on=keys, how='left', indicator=True)
    inserts_df = inserts_df[inserts_df['_merge'] == 'both'].reset_index(drop=True)
    inserts_df = inserts_df.drop(columns=['_merge'])

    inserts_df = _get_stamped_capture(inserts_df, 'INSERT', source)  
    return inserts_df

get_wide_inserts_df(curr_df.copy(), last_df.copy(), KEYS, SOURCE)

,policyID,statecode,county,eq_site_limit,hu_site_limit,fl_site_limit,fr_site_limit,tiv_2011,tiv_2012,eq_site_deductible,...,fl_site_deductible,fr_site_deductible,point_latitude,point_longitude,line,construction,point_granularity,capture_type,capture_source,capture_datetime


In [229]:
def get_long_inserts_df(curr_df, last_df, keys, source):
    
    wide_inserts_df = get_wide_inserts_df(curr_df, last_df, keys, source)
    non_keys = list(set(curr_df.columns.tolist()) - set(keys))
    inserts_df = wide_inserts_df.melt(id_vars=keys, value_vars=non_keys, var_name='column_name', value_name='value_stored')

    inserts_df = _get_stamped_capture(inserts_df, 'INSERT', source)  
    return inserts_df

get_long_inserts_df(curr_df.copy(), last_df.copy(), KEYS, SOURCE)

,policyID,column_name,value_stored,capture_type,capture_source,capture_datetime


In [230]:
def get_wide_deletes_df(curr_df, last_df, keys, source):
    
    deletes_df = pd.merge(curr_df, last_df, on=keys, how='outer', indicator=True)
    deletes_df = deletes_df[deletes_df['_merge'] == 'right_only'].reset_index(drop=True)
    deletes_df = deletes_df[keys]

    deletes_df = pd.merge(deletes_df, last_df, on=keys, how='left', indicator=True)
    deletes_df = deletes_df[deletes_df['_merge'] == 'both'].reset_index(drop=True)
    deletes_df = deletes_df.drop(columns=['_merge'])

    deletes_df = _get_stamped_capture(deletes_df, 'DELETE', source)  
    return deletes_df

get_wide_deletes_df(curr_df.copy(), last_df.copy(), KEYS, SOURCE)

,policyID,statecode,county,eq_site_limit,hu_site_limit,fl_site_limit,fr_site_limit,tiv_2011,tiv_2012,eq_site_deductible,...,fl_site_deductible,fr_site_deductible,point_latitude,point_longitude,line,construction,point_granularity,capture_type,capture_source,capture_datetime


In [231]:
def get_long_deletes_df(curr_df, last_df, keys, source):
    
    wide_deletes_df = get_wide_deletes_df(curr_df, last_df, keys, source)
    non_keys = list(set(curr_df.columns.tolist()) - set(keys))
    deletes_df = wide_deletes_df.melt(id_vars=keys, value_vars=non_keys, var_name='column_name', value_name='value_stored')

    deletes_df = _get_stamped_capture(deletes_df, 'DELETE', source)    
    return deletes_df

get_long_deletes_df(curr_df.copy(), last_df.copy(), KEYS, SOURCE)

,policyID,column_name,value_stored,capture_type,capture_source,capture_datetime


In [232]:
def get_wide_updates_df(curr_df, last_df, keys, source):

    same_row_df = pd.merge(curr_df, last_df, on=last_df.columns.tolist(), how='outer', indicator=True)
    same_row_df = same_row_df[same_row_df['_merge'] == 'both'].reset_index(drop=True)
    same_row_df = same_row_df[keys]

    same_keys_df = pd.merge(curr_df, last_df, on=keys, how='outer', indicator=True)
    same_keys_df = same_keys_df[same_keys_df['_merge'] == 'both'].reset_index(drop=True)
    same_keys_df = same_keys_df[keys]

    updates_df = pd.merge(same_row_df, same_keys_df, on=keys, how='outer', indicator=True)
    updates_df = updates_df[updates_df['_merge'] == 'right_only'].reset_index(drop=True)
    updates_df = updates_df.drop(columns=['_merge'])

    updates_df = pd.merge(updates_df, curr_df, on=keys, how='left', indicator=True)
    updates_df = updates_df[updates_df['_merge'] == 'both'].reset_index(drop=True)
    updates_df = updates_df.drop(columns=['_merge'])

    updates_df = _get_stamped_capture(updates_df, 'UPDATE', source)  
    return updates_df

get_wide_updates_df(curr_df.copy(), last_df.copy(), KEYS, SOURCE)

,policyID,statecode,county,eq_site_limit,hu_site_limit,fl_site_limit,fr_site_limit,tiv_2011,tiv_2012,eq_site_deductible,...,fl_site_deductible,fr_site_deductible,point_latitude,point_longitude,line,construction,point_granularity,capture_type,capture_source,capture_datetime
0,708726,FL,PINELLAS COUNTY,0,717846.3,0,717846.3,717846.3,1097387.43,0,...,0,0,28.122885,-82.770218,Residential,Masonry,3,UPDATE,SYSTEM-01,2021-10-31 09:22:52


In [233]:
def get_long_updates_df(curr_df, last_df, keys, source):
    
    wide_updates_df = get_wide_updates_df(curr_df, last_df, keys, source)
    non_keys = list(set(curr_df.columns.tolist()) - set(keys))
    new_updates_df = wide_updates_df.melt(id_vars=keys, value_vars=non_keys, var_name='column_name', value_name='value_stored')
    old_updates_df = last_df.melt(id_vars=keys, value_vars=non_keys, var_name='column_name', value_name='value_stored')
    
    updates_df = pd.merge(new_updates_df, old_updates_df, on=new_updates_df.columns.tolist(), how='outer', indicator=True)
    updates_df = updates_df[updates_df['_merge'] == 'left_only'].reset_index(drop=True)
    updates_df = updates_df.drop(columns=['_merge'])

    updates_df = _get_stamped_capture(updates_df, 'UPDATE', source)    
    return updates_df

get_long_updates_df(curr_df.copy(), last_df.copy(), KEYS, SOURCE)

,policyID,column_name,value_stored,capture_type,capture_source,capture_datetime
0,708726,county,PINELLAS COUNTY,UPDATE,SYSTEM-01,2021-10-31 09:22:53


In [234]:
def has_changed(curr_df, last_df):
    
    changes_df = pd.merge(curr_df, last_df, on=curr_df.columns.tolist(), how='outer', indicator=True)
    changes_df = changes_df[(changes_df['_merge'] == 'left_only')|(changes_df['_merge'] == 'right_only')].reset_index(drop=True)
    if(len(changes_df) > 0):
        return True
    return False

has_changed(curr_df, last_df)

True

In [235]:
curr_df.to_csv('last.csv', index=False, sep=',')

